# HouseBrain Architect - Fine-Tuning on Colab

This notebook fine-tunes the `Qwen/Qwen2.5-3B-Instruct` model on our curated house plan dataset.

**Instructions:**

1.  **Set up the GPU:** Go to `Runtime` > `Change runtime type` and select `T4 GPU` or another available GPU.
2.  **Run each cell in order:** Execute the cells one by one from top to bottom.
3.  **Hugging Face Token (Optional but Recommended):** The second code cell will ask for a Hugging Face token. While not always required for public models, it's good practice. You can create a token in your Hugging Face account settings and add it as a "Secret" in Colab (click the 🔑 icon on the left). Name the secret `HF_TOKEN`.
4.  **Download the result:** After the final cell runs, a file named `housebrain_v1_adapters.zip` will appear in the file browser on the left. Right-click it and select `Download` to save your trained model adapters.


In [ ]:
# --- 1. Clone the Repository ---
# This will download your project files, including the necessary datasets.

!git clone https://github.com/Vinay-O/HouseBrainLLM.git
%cd HouseBrainLLM

print("✅ Repository cloned and directory changed.")


In [ ]:
# --- 2. Install Dependencies ---
# This multi-step process ensures that all libraries, especially torch, bitsandbytes,
# and triton, are installed and compiled correctly for the Colab A100 GPU environment.

# Step A: Install the main application libraries
!pip install -U "transformers==4.41.2" "peft==0.10.0" "accelerate==0.30.1" "datasets==2.19.1"

# Step B: Force a re-install of torch and a compatible triton from the PyTorch CUDA 12.1 index
# This is the critical step for A100 GPUs.
!pip install -U --force-reinstall "torch==2.3.1" "triton==2.3.1" --index-url https://download.pytorch.org/whl/cu121

# Step C: Install the correct version of bitsandbytes
!pip install -U "bitsandbytes==0.43.1"


In [ ]:
# --- 3. Imports & Login ---

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig
import os

# Optional: Login to Hugging Face
from huggingface_hub import login
from google.colab import userdata

try:
  hf_token = userdata.get('HF_TOKEN')
  login(token=hf_token)
  print("✅ Successfully logged into Hugging Face.")
except Exception as e:
  print("Proceeding without Hugging Face login. Add a secret named HF_TOKEN if you encounter download issues.")


In [ ]:
# --- 4. Load and Prepare Dataset ---

model_name = "Qwen/Qwen2.5-3B-Instruct"

# The tokenizer is loaded first to be used in the data preparation.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define the dataset file we want to use
dataset_file = "finetune_dataset_senior_architect_v3.jsonl"

# Load the dataset from the cloned repository
# For the Senior Architect, we'll train on the entire dataset and not use a separate validation set for now.
full_dataset = load_dataset("json", data_files=dataset_file, split="train")


# This function formats the prompts and tokenizes them for the Senior Architect
def formatting_and_tokenizing_func(examples):
    texts = []
    # The dataset is already in the OpenAI "messages" format.
    # We just need to apply the chat template.
    for conversation in examples["messages"]:
        # CRITICAL: Update the system prompt to be extremely direct for the correction task
        system_prompt_updated = {
            "role": "system",
            "content": "You are an expert AI architect. Your only task is to review a flawed house plan and output the corrected version as a single, raw JSON object. Do not add any conversational text or markdown."
        }
        
        # Find the user and assistant messages
        user_msg = next((msg for msg in conversation if msg['role'] == 'user'), None)
        assistant_msg = next((msg for msg in conversation if msg['role'] == 'assistant'), None)

        if user_msg and assistant_msg:
            # Reconstruct the conversation with the updated, stricter system prompt
            templated_chat = tokenizer.apply_chat_template(
                [system_prompt_updated, user_msg, assistant_msg],
                tokenize=False,
                add_generation_prompt=False
            )
            texts.append(templated_chat)

    # Tokenize the formatted text
    return tokenizer(texts, truncation=True, max_length=2048, padding=False) # Increased max_length for complex plans


# Apply the function to the dataset
processed_dataset = full_dataset.map(formatting_and_tokenizing_func, batched=True, remove_columns=full_dataset.column_names)

print("✅ Senior Architect dataset loaded and prepared.")
print(f"Total training samples: {len(processed_dataset)}")


In [ ]:
# --- 5. Configure and Load Model ---

# Configure 4-bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the base model with our quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
)

# Configure LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

print("✅ Model configured with 4-bit quantization and LoRA.")
model.print_trainable_parameters()


In [ ]:
# --- 6. Start Fine-Tuning ---

output_dir = "housebrain_senior_architect_v3_adapters"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=25, # Log every 25 steps
    num_train_epochs=2, # Two epochs should be sufficient with the larger dataset
    save_strategy="epoch", # Save at the end of each epoch
    fp16=True, # Use fp16 for mixed-precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset, # Use our fully processed dataset
    # eval_dataset is removed as we are not evaluating during this training run
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("🚀 Starting Senior Architect (v3) training...")
trainer.train()

print("✅ Fine-tuning complete.")
trainer.save_model(output_dir)
print(f"💾 Final model adapters saved to '{output_dir}'.")


In [ ]:
# --- 7. Package and Download ---
import shutil

# Zip the adapters folder
archive_path = shutil.make_archive("housebrain_senior_architect_v3_adapters", 'zip', "housebrain_senior_architect_v3_adapters")

print(f"✅ Model adapters saved and zipped at: {archive_path}")
print("You can now download 'housebrain_senior_architect_v3_adapters.zip' from the Files panel on the left.")
